# Apppendix A: Computer Code and Results

In [1]:
# Import libraries  
import numpy as np
import pandas as pd

# 1) Edit Distance

In [2]:
# Using matrix dynamic programming, define a function which given string 1, string 2, 
# the substitution cost, the deletion cost and the insertion cost, we calculate the edit distance
def edit_distance(string1, string2, sub_cost=1, del_cost=1, ins_cost=1):
    # Construct an initial m x n matrix containing zeros
    m = len(string1) + 1; n = len(string2) + 1; matrix = np.zeros((m, n))
    # Fill the matrix for the base case, where one input string is the empty string
    matrix[::,0] = [x * del_cost for x in range(m)]
    matrix[0,::] = [y * ins_cost for y in range(n)]
    # Iterate over range n and for each m skipping the first value, e.g. the empty string case
    for x in range(1, m): 
        for y in range(1, n):
            # If the letters do not match, select the operation with the minimum cost
            sub = sub_cost if string1[x-1] != string2[y-1] else 0
            matrix[x,y] = min(matrix[x-1, y-1] + sub, matrix[x-1, y] + del_cost, matrix[x, y-1] + ins_cost)
    distance = int(matrix[m - 1, n - 1])
    # Recover the type of operations performed by examination of the path within the matrix
    # from the bottom right [m,n] to the upper left [0,0]
    row = m-1; column = n-1; operations = []
    while not ((row == 0) and (column == 0)):
        # If we haven't reached the first row or the first column
        if (row != 0) & (column != 0): 
            # Value of left cell is smaller than the cell above and diagonal, shift left 
            if (matrix[row,column-1] + ins_cost < matrix[row-1,column] + del_cost) \
            & (matrix[row,column-1] + ins_cost < matrix[row-1,column-1] + sub_cost): 
                column = column-1; operations.append('I')
            # Value of above cell is smaller than cell to the left and diagonal, shift upward
            elif (matrix[row-1,column] + del_cost < matrix[row,column-1] + ins_cost) \
            & (matrix[row-1,column] + del_cost < matrix[row-1,column-1] + sub_cost): 
                row = row-1; operations.append('D')
            # Value of diagonal cell is smaller or equal than the cell left and above, shift diagonal
            elif (matrix[row-1,column-1] + sub_cost <= matrix[row,column-1] + ins_cost) \
            & (matrix[row-1,column-1] + sub_cost <= matrix[row-1,column] + del_cost): 
                if matrix[row-1,column-1] == matrix[row,column] - sub_cost: 
                    row = row-1; column = column-1; operations.append('S')
                else: 
                    row = row-1; column = column-1; operations.append('-')
        # If we have reached to the first row, shift left
        elif column != 0: 
            column = column-1; operations.append('I')
        # If we have reached to the first column, shift upwards
        else: 
            row = row-1; operations.append('D')
    operations = list(reversed(operations))
    return distance, operations

In [3]:
# First test: DNA
X = "ACTACTAGATTACTTACGGATCAGGTACTTTAGAGGCTTGCAACCA"
Y = "TACTAGCTTACTTACCCATCAGGTTTTAGAGATGGCAACCA"

In [4]:
# Print the results from DNA without penalty
distance, operations = edit_distance(X,Y)
print('Edit distance without penalty: ', distance)
print ('Optimal sequence without penalty: ', operations)

Edit distance without penalty:  10
Optimal sequence without penalty:  ['D', 'D', '-', '-', '-', '-', '-', '-', 'S', '-', '-', '-', '-', '-', '-', '-', '-', 'S', 'S', '-', '-', '-', '-', '-', '-', '-', 'D', 'D', '-', '-', '-', '-', '-', '-', '-', 'D', 'S', '-', 'S', '-', '-', '-', '-', '-', '-', '-']


In [5]:
# Print the results from DNA with penalty
distance, operations = edit_distance(X,Y,1,2,2)
print('Edit distance with penalty: ', distance)
print ('Optimal sequence with penalty: ', operations)

Edit distance with penalty:  15
Optimal sequence with penalty:  ['D', 'D', '-', '-', '-', '-', '-', '-', 'S', '-', '-', '-', '-', '-', '-', '-', '-', 'S', 'S', '-', '-', '-', '-', '-', '-', '-', 'D', 'D', '-', '-', '-', '-', '-', '-', '-', 'D', 'S', '-', 'S', '-', '-', '-', '-', '-', '-', '-']


In [6]:
# Second test: Proteins
X = "AASRPRSGVPAQSDSDPCQNLAATPIPSRPPSSQSCQKCRADARQGRWGP"
Y = "SGAPGQRGEPGPQGHAGAPGPPGPPGSDG"

In [7]:
# Print the results from Proteins without penalty
distance, operations = edit_distance(X,Y)
print('Edit distance without penalty: ', distance)
print ('Optimal sequence without penalty: ', operations)

Edit distance without penalty:  37
Optimal sequence without penalty:  ['D', 'D', '-', 'D', 'D', 'D', 'D', '-', 'S', '-', 'S', '-', 'S', 'S', 'S', 'S', 'S', 'S', '-', 'S', 'S', '-', 'S', 'S', '-', 'S', '-', 'S', 'S', '-', '-', 'S', '-', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', '-', 'D', 'D', 'D', '-', 'D', 'D', 'D', 'D']


In [8]:
# Print the results from Proteins with penalty
distance, operations = edit_distance(X,Y,1,2,2)
print('Edit distance with penalty: ', distance)
print ('Optimal sequence with penalty: ', operations)

Edit distance with penalty:  58
Optimal sequence with penalty:  ['D', 'D', '-', 'D', 'D', 'D', 'D', '-', 'S', '-', 'S', '-', 'S', 'S', 'S', 'S', 'S', 'S', '-', 'S', 'S', '-', 'S', 'S', '-', 'S', '-', 'S', 'S', '-', '-', 'S', '-', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', '-', 'D', 'D', 'D', '-', 'D', 'D', 'D', 'D']


# 2) Huffman codes

In [9]:
# Function to explore the Huffman tree and extract the corresponding binary code recursivly
def assign_code(nodes, label, result, prefix = ''): 
    childs = nodes[label]; tree = {}
    # As long as there are 2 children below one node explore the Huffman tree recursively
    if len(childs) == 2: 
        tree['0'] = assign_code(nodes, childs[0], result, prefix+'0')
        tree['1'] = assign_code(nodes, childs[1], result, prefix+'1')
        return tree
    # If there are no children return the binary prefix code for this elementary node
    else:
        result[label] = prefix 
        return label

In [10]:
# Encode a specific text T in binary format
def get_prefix(T):
    nodes = {};code = {};tree = {}
    T = T.lower() 
    alphabet = list(set(T)) 
    freq = {x : T.count(x) for x in alphabet} 
    for n in freq.keys(): 
        # Initialize nodes for all items in the alphabet with no children
        nodes[n] = []
    while len(freq) > 1: 
        # Sort the dictionary by values
        val = sorted(freq.items(), key=lambda x:x[1]) 
        # Extract keys with the two lowest value in the frequency dictionary
        v1 = val[0][0] 
        v2 = val[1][0] 
        # Update frequency dictionary: remove entries with the two lowest values, add value for new node
        freq[v1 + v2] = freq.pop(v1) + freq.pop(v2) 
        # add newly created node with the corresponding two children
        nodes[v1 + v2] = [v1, v2] 
    # Set root node for the start of the analysis of the hoffman tree
    root = v1 + v2
    # Call function to recursivly extract the codes and the tree
    tree = assign_code(nodes, root, code)
    return ''.join([code[char] for char in T]), code, tree, {x : T.count(x) for x in alphabet} 

In [11]:
# Decode a specific text T from binary format
def get_T(binary, tree):
    decoded = []; i = 0
    # Check the tree key elements of the dictionry by adding binarys from the prefix iteratively
    while i < len(binary):
        ch = binary[i]  
        act = tree[ch]
        while not isinstance(act, str):
            i += 1
            ch = binary[i]  
            act = act[ch]        
        # If the value element of the tree dictionary is a character append it to the solution
        decoded.append(act);i += 1
    return ''.join(decoded)

In [12]:
# First input text
Hamlet = 'O all you host of heaven! O earth! What else? And shall I couple hell? Oh, fie! Hold, hold, my heart, \
And you, my sinews, grow not instant old, But bear me stiffly up. Remember thee! Ay, thou poor ghost, whiles \
memory holds a seat In this distracted globe. Remember thee! Yea, from the table of my memory I’ll wipe away all \
trivial fond records, All saws of books, all forms, all pressures past That youth and observation copied there, \
And thy commandment all alone shall live Within the book and volume of my brain, Unmixed with baser matter. \
Yes, by heaven! O most pernicious woman! O villain, villain, smiling, damned villain! My tables! Meet it is I \
set it down That one may smile, and smile, and be a villain. At least I’m sure it may be so in Denmark. So, \
uncle, there you are. Now to my word.'

In [13]:
prefix1, code1, tree1, freq1 = get_prefix(Hamlet) # Get the prefix code the tree and frequencies for the first text
prefix1

'100100101110001000000100110011010100011000100111111011000100101000000110001110101101000111101111011010110010010011101011110010110110001101011001010011100010110110001110100011111111010100000001011111100111000111111100010111000100000010100101000110011010101101010100011100011000111010001000101000000010011100001111000100000101111011010110011000100110000111001111001100010011000011100111100110110100100110001110101111001011001111001011111100111000010011001101010011110011011010010011111010111110111010100111111011110010101111110011001101001001111010010110000101111101111101101011111100110001001100001110011110011010010101001100011010011101011110010011011111000111110110010101000001000010000100100101010110101010101100011001111011011111011011110100111011001000110110001110111011010110010110100101111000110110001001101010001101010100110011100100101011111100010011111101100111100101001110000101100011101111100110111110110111001110010100100110001001100001110111110010110011111111010110110000101111100001101

In [14]:
get_T(prefix1, tree1) # Decode the first input text

'o all you host of heaven! o earth! what else? and shall i couple hell? oh, fie! hold, hold, my heart, and you, my sinews, grow not instant old, but bear me stiffly up. remember thee! ay, thou poor ghost, whiles memory holds a seat in this distracted globe. remember thee! yea, from the table of my memory i’ll wipe away all trivial fond records, all saws of books, all forms, all pressures past that youth and observation copied there, and thy commandment all alone shall live within the book and volume of my brain, unmixed with baser matter. yes, by heaven! o most pernicious woman! o villain, villain, smiling, damned villain! my tables! meet it is i set it down that one may smile, and smile, and be a villain. at least i’m sure it may be so in denmark. so, uncle, there you are. now to my word.'

In [15]:
table1 = pd.concat([pd.Series(code1), pd.Series(freq1)], axis=1, sort= True) 
table1.columns = ['Code', 'Frequency'] # Preparation for the final overview table 

In [16]:
# Second input text
Faust = "Habe nun, ach! Philosophie, Juristerei und Medizin, Und leider auch Theologie Durchaus studiert, mit \
heissem Bemühn. Da steh ich nun, ich armer Tor! Und bin so klug als wie zuvor; Heisse Magister, heisse Doktor \
gar Und ziehe schon an die zehen Jahr Herauf, herab und quer und krumm Meine Schüler an der Nase herum Und sehe, \
dass wir nichts wissen können! Das will mir schier das Herz verbrennen. Zwar bin ich gescheiter als all die Laffen, \
Doktoren, Magister, Schreiber und Pfaffen; Mich plagen keine Skrupel noch Zweifel, Fürchte mich weder vor Hölle \
noch Teufel Dafür ist mir auch alle Freud entrissen, Bilde mir nicht ein, was Rechts zu wissen, Bilde mir nicht \
ein, ich könnte was lehren, Die Menschen zu bessern und zu bekehren. Auch hab ich weder Gut noch Geld, Noch Ehr \
und Herrlichkeit der Welt; Es möchte kein Hund so länger leben! Drum hab ich mich der Magie ergeben, Ob mir durch \
Geistes Kraft und Mund Nicht manch Geheimnis würde kund; Dass ich nicht mehr mit saurem Schweiss Zu sagen brauche, \
was ich nicht weiss; Dass ich erkenne, was die Welt Im Innersten zusammenhält, Schau alle Wirkenskraft und Samen, \
Und tu nicht mehr in Worten kramen."

In [17]:
prefix2, code2, tree2, freq2 = get_prefix(Faust) # Get the prefix code the tree and frequencies for the second text
prefix2

'100000010110100101111011000010111101011110001110011000001011011111101000010001010011001100000011110000110100001000101001011010111111010011000000111101000111001101001110101010111000010111101111110010010110111010001000101010111101011110000101111011111011000101010110110100111111000100001100110001111001110000101100000110011000000101010100101111101100000111110011000000100000011111001110011000011011101001001111001111010111110010101010011111100001010100011001101010010111011010010100101101001010001011001011101111101100011110011100110101000111101011001100011110110000101111010111110101100110001110001011110010010011111110011110000011100101101111000010111101111101101010101011111001111000011101101101100000000101011100010110000111111100011010010111001000000011010011111000001111101000111110000101010001100110101111001000010010101010001110011010011111010111110000101010001100110101111101111000001101110011110000011111100101000010111111000010111101111100100010100101000010111001111001100011000010111110001

In [18]:
get_T(prefix2, tree2) # Decode the second input text

'habe nun, ach! philosophie, juristerei und medizin, und leider auch theologie durchaus studiert, mit heissem bemühn. da steh ich nun, ich armer tor! und bin so klug als wie zuvor; heisse magister, heisse doktor gar und ziehe schon an die zehen jahr herauf, herab und quer und krumm meine schüler an der nase herum und sehe, dass wir nichts wissen können! das will mir schier das herz verbrennen. zwar bin ich gescheiter als all die laffen, doktoren, magister, schreiber und pfaffen; mich plagen keine skrupel noch zweifel, fürchte mich weder vor hölle noch teufel dafür ist mir auch alle freud entrissen, bilde mir nicht ein, was rechts zu wissen, bilde mir nicht ein, ich könnte was lehren, die menschen zu bessern und zu bekehren. auch hab ich weder gut noch geld, noch ehr und herrlichkeit der welt; es möchte kein hund so länger leben! drum hab ich mich der magie ergeben, ob mir durch geistes kraft und mund nicht manch geheimnis würde kund; dass ich nicht mehr mit saurem schweiss zu sagen bra

In [19]:
table2 = pd.concat([pd.Series(code2), pd.Series(freq2)], axis=1, sort=True)
table2.columns = ['Code', 'Frequency'] # Preparation for the final overview table 

In [20]:
# Create table for an overview over |c(x)|, frequencies and a comparision to the regular binary representation
table = pd.merge(table1, table2, how='outer', left_index=True, right_index=True, sort=True, 
                 suffixes=('_Hamlet', '_Faust'), copy=True)
table['Regular Binary'] = table.index.map(lambda x: ''.join(format(ord(x), 'b')))
table['Both Codes < Binary'] = table.apply(lambda row: (len(str(row['Code_Hamlet'])) <= 
                                                        len(str(row['Regular Binary'])) and 
                                                  len(str(row['Code_Faust'])) <= 
                                                        len(str(row['Regular Binary']))), axis=1)
table.sort_values('Frequency_Hamlet', ascending=False)

,Code_Hamlet,Frequency_Hamlet,Code_Faust,Frequency_Faust,Regular Binary,Both Codes < Binary
,00,149.0,111,197.0,100000,True
e,1110,68.0,010,131.0,1100101,True
a,1011,55.0,0001,49.0,1100001,True
o,1001,52.0,110000,20.0,1101111,True
l,1000,48.0,01100,32.0,1101100,True
t,0110,46.0,10011,38.0,1110100,True
i,0101,42.0,1010,77.0,1101001,True
s,11111,38.0,0011,65.0,1110011,True
n,11110,36.0,1011,83.0,1101110,True
m,11011,34.0,10010,35.0,1101101,True


In [21]:
# Calculating C for different variations and cross-check results
print(np.nansum(table.apply(lambda row: (len(str(row['Code_Faust'])) * row['Frequency_Faust']), axis=1)) / \
np.nansum(table.apply(lambda row: (len(str(row['Regular Binary'])) * row['Frequency_Faust']), axis=1)))
print(np.nansum(table.apply(lambda row: (len(str(row['Code_Hamlet'])) * row['Frequency_Hamlet']), axis=1)) / \
np.nansum(table.apply(lambda row: (len(str(row['Regular Binary'])) * row['Frequency_Hamlet']), axis=1)))

0.6241089613034623
0.6276536828502861


### Brief comment on the results
At first glance, our results show that almost all individual letters have a simpler binary representation than the standard notation. This is especially true for the most used letters and is an indicator that our implementation of the Huffman coding produces compressed data without any loss of information. In the next step we have a more formal validation of the observed results.

A prefix code is optimal if the following defined C is minimal among all prefix codes:

$$ C = \sum _ { x \in A_H } f _ { x } | c ( x ) | $$

Here, we can calculate $C_{Hamlet_p}$ for the prefix code of the first input text and $C_{Faust_p}$ for the prefix code of the second input text with the following results: 

$$ C_{Hamlet_p} = 3,400 \quad \textrm{and} \quad C_{Faust_p} = 4,903 $$

According to our proof of corectness these values for C must be minimal. We can quickly check against the standard binary representation of characters and we obtain the following: 

$$ C_{Hamlet_s} = 5,417 \quad \textrm{and} \quad C_{Faust_s} = 7,856 $$

In both cases we could reduce the number of bits required to store the texts by 37 %.